In [ ]:
import get_price.get_futures_price as gfp
panel_data = gfp.get_futures_price_preloaded(ticker_head='CL')
unique_dates = panel_data['settle_date'].unique()

data4day = panel_data[panel_data['settle_date']==unique_dates[4000]]


In [ ]:
import get_price.curve_data as cd
rolling_data = cd.get_rolling_curve_data(ticker_head='CL',num_contracts=15,
                                         front_tr_dte_limit=10,
                                        date_from=20050101,
                                        date_to=20160112)



In [23]:
import opportunity_constructs.spread_carry as sc
#spread_carry_output = sc.generate_spread_carry_sheet_4date(report_date=20160120)
#spread_report = spread_carry_output['spread_report']
#ticker_head_list = list(spread_report['ticker_head'].unique())
# SB might be a good buy but cannot trade on 6
# KC on 7 might be a good trade 
# work on 1 lot LN Z-G on 8
# CT might be a good buy on 9
# adjust yor 2 C buy order if you get exexuted in S

spread_report[spread_report['ticker_head']==ticker_head_list[20]]


IndexError: list index out of range

In [2]:
import risk.historical_risk as hr
import pandas as pd
import opportunity_constructs.utilities as opUtil
import contract_utilities.contract_meta_info as cmi
wuhu = hr.get_historical_risk_4strategy(alias='spreadCarryJan16')
wuhu






-644.2750000000201

In [4]:
butterfly_followup_frame = pd.DataFrame(results)

In [1]:
import formats.futures_strategy_formats as ff
import ta.prepare_daily as tpd
tpd.prepare_curve_pca()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(yield_history[5].values)
plt.grid()               
plt.show()


In [ ]:
import contract_utilities.contract_meta_info as cmi
cmi.get_contract_specs(wuhu['ticker1'][0])['ticker_head']

In [ ]:
residuals = yield_data-pca_out['model_fit']
residuals

In [ ]:
import backtesting.backtest_curve_pca as bcp

pnl_path = []

unique_dates = rolling_data[0].index

for i in range(len(unique_dates)):
    
    pnl_path.append(bcp.backtest_curve_pca_4date(residuals=residuals[i,:],
                                                 change_data=change5_data[i,:],
                                                 tr_dte_data = tr_dte_data[i,:]))







In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_fit = pca.fit(yield_data)
pca_fit.

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
plt.plot(np.cumsum(pnl_path))
plt.grid()               
plt.show()

In [ ]:
import opportunity_constructs.curve_pca as cpc
import pandas as pd
#report_out = cpc.get_curve_pca_report(ticker_head='CL', date_to=20160112 )

report_out['factor_load1']







In [ ]:
np.mean(pnl_path)

In [ ]:
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10
ax.plot(class1_sample[0,:], class1_sample[1,:], class1_sample[2,:],
        'o', markersize=8, color='blue', alpha=0.5, label='class1')
ax.plot(class2_sample[0,:], class2_sample[1,:], class2_sample[2,:],
        '^', markersize=8, alpha=0.5, color='red', label='class2')

plt.title('Samples for class 1 and class 2')
ax.legend(loc='upper right')

plt.show()

In [ ]:
all_samples = np.concatenate((class1_sample, class2_sample), axis=1)
assert all_samples.shape ==(3,40), "The matrix has not the dimensions 3x40"

In [ ]:
mean_x = np.mean(all_samples[0,:])
mean_y = np.mean(all_samples[1,:])
mean_z = np.mean(all_samples[2,:])

mean_vector = np.array([[mean_x],[mean_y],[mean_z]])
print('Mean Vector:\n', mean_vector)






In [ ]:
scatter_matrix = np.zeros((3,3))

for i in range(all_samples.shape[1]):
    scatter_matrix +=(all_samples[:,i].reshape(3,1)).dot(
    (all_samples[:,i].reshape(3,1)-mean_vector).T)
print('Scatter Matrix\n',scatter_matrix)    

In [ ]:
cov_mat = np.cov([all_samples[0,:], all_samples[1,:],all_samples[2,:]])
print('Covariance Matrix:\n', cov_mat)

In [ ]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)

for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i].reshape(1,3).T
    eigvec_cov = eig_vec_cov[:,i].reshape(1,3).T
    assert eigvec_sc.all() == eigvec_cov.all(), 'Eigenvectors are not identical'

    print('Eigenvector {}: \n{}'.format(i+1, eigvec_sc))
    print('Eigenvalue {} from scatter matrix: {}'.format(i+1, eig_val_sc[i]))
    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
    print('Scaling factor: ', eig_val_sc[i]/eig_val_cov[i])
    print(40 * '-')

In [ ]:
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv),
                                         eig_val_sc[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from matplotlib.patches import FancyArrowPatch


class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(all_samples[0,:], all_samples[1,:], all_samples[2,:],
        'o', markersize=8, color='green', alpha=0.2)
ax.plot([mean_x], [mean_y], [mean_z],
        'o', markersize=10, color='red', alpha=0.5)
for v in eig_vec_sc.T:
    a = Arrow3D([mean_x, v[0]+mean_x],
                [mean_y, v[1]+mean_y],
                [mean_z, v[2]+mean_z],
                mutation_scale=20, lw=3, arrowstyle="-|>", color="r")

    ax.add_artist(a)
ax.set_xlabel('x_values')
ax.set_ylabel('y_values')
ax.set_zlabel('z_values')

plt.title('Eigenvectors')

plt.show()
multi_columns = pd.MultiIndex.from_product([[ticker1, ticker2, ticker3],data_1.columns.values], names=['first', 'second'])


In [ ]:
transformed = matrix_w.T.dot(all_samples)
assert transformed.shape == (2,40), "The matrix is not 2x40 dimensional."
transformed
matrix_w

In [ ]:
import matplotlib.pyplot as plt
plt.plot(merged_dataframe.index,(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-
         merged_dataframe['BOK2016']['close_price'])
         
         
plt.show()

In [ ]:
(merged_dataframe['BOH2016']['close_price']+merged_dataframe['BON2016']['close_price'])/2-merged_dataframe['BOK2016']['close_price']

In [ ]:


import matplotlib.pyplot as plt
plt.plot(range(len(aligned_dataframe.index)),aligned_dataframe['c1']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c2']['tr_dte'],
         range(len(aligned_dataframe.index)),aligned_dataframe['c3']['tr_dte'])
plt.grid()         
         
plt.show()

    

    

In [ ]:
merged_dataframe['c1']['tr_dte'].isnull().sum()
merged_dataframe['c3']['tr_dte'].isnull().sum()

In [ ]:
futures_dataframe

In [ ]:
print(trDte1)
print(trDte2)
print(trDte3)

In [ ]:
bf_price = (aligned_dataframe['c1']['close_price']+aligned_dataframe['c3']['close_price'])/2-aligned_dataframe['c2']['close_price']

In [ ]:
plt.plot(range(len(aligned_dataframe.index)),bf_price)
plt.grid()         
         
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(price_ratio)
plt.grid()
plt.show()

In [ ]:
import numpy as np
nan_matrix = np.empty((2,2))
nan_matrix[:] = np.NAN
nan_matrix